# Import pixiedust
Start by importing pixiedust which if all bootstrap and install steps were run correctly.
You should see below for opening the pixiedust database successfully with no errors.
Depending on the version of pixiedust that gets installed, it may ask you to update.
If so, run this first cell.

In [ ]:
#!pip install --user --upgrade pixiedust
#!pip install pandas 

In [15]:
import pixiedust

▸,:,


In [16]:
pixiedust.enableJobMonitor()

▸,:,


Spark Job Progress Monitor already enabled


# Creating the SQLContext and inspecting pyspark Context
Pixiedust imports pyspark and the SparkContext + SparkSession should be already available through the "sc" and "spark" variables respectively.

In [17]:
# Print Spark info and create sql_context
print('Spark Version: {0}'.format(sc.version))
print('Python Version: {0}'.format(sc.pythonVer))
print('Application Name: {0}'.format(sc.appName))
print('Application ID: {0}'.format(sc.applicationId))
print('Spark Master: {0}'.format( sc.master))

sql_context = SQLContext(sc, sparkSession=spark)

▸,:,


Spark Version: 2.2.0
Python Version: 3.5
Application Name: pyspark-shell
Application ID: application_1509968683912_0002
Spark Master: yarn


# Download and ingest the GPX data

In [20]:
%%bash
s3-dist-cp -D mapreduce.task.timeout=60000000 --src=s3://geowave-gpx-data/gpx --dest=hdfs://$HOSTNAME:8020/tmp/ 

▸,:,


17/11/06 12:30:10 INFO s3distcp.S3DistCp: Running with args: -libjars /usr/share/aws/emr/s3-dist-cp/lib/guava-15.0.jar,/usr/share/aws/emr/s3-dist-cp/lib/s3-dist-cp-2.6.0.jar,/usr/share/aws/emr/s3-dist-cp/lib/s3-dist-cp.jar -D mapreduce.task.timeout=60000000 --src=s3://geowave-gpx-data/gpx --dest=hdfs://ip-10-0-0-224:8020/tmp/ 
17/11/06 12:30:10 INFO s3distcp.S3DistCp: S3DistCp args: --src=s3://geowave-gpx-data/gpx --dest=hdfs://ip-10-0-0-224:8020/tmp/ 
17/11/06 12:30:10 INFO s3distcp.S3DistCp: Using output path 'hdfs:/tmp/b098455b-de8c-4a2e-a0ed-6a7185672ba3/output'
17/11/06 12:30:10 INFO s3distcp.S3DistCp: GET http://169.254.169.254/latest/meta-data/placement/availability-zone result: us-east-1f
17/11/06 12:30:13 INFO s3distcp.FileInfoListing: Opening new file: hdfs:/tmp/b098455b-de8c-4a2e-a0ed-6a7185672ba3/files/1
17/11/06 12:30:13 INFO s3distcp.S3DistCp: Created 1 files to copy 66 files 
17/11/06 12:30:13 INFO s3distcp.S3DistCp: Reducer number: 31
17/11/06 12:30:14 INFO impl.Timelin

In [21]:
%%bash
/opt/accumulo/bin/accumulo shell -u root -p secret -e "importtable geowave.germany_gpx_SPATIAL_IDX /tmp/spatial"
/opt/accumulo/bin/accumulo shell -u root -p secret -e "importtable geowave.germany_gpx_GEOWAVE_METADATA /tmp/metadata"

▸,:,


2017-11-06 12:31:22,576 [conf.ConfigSanityCheck] WARN : Use of instance.dfs.uri and instance.dfs.dir are deprecated. Consider using instance.volumes instead.
2017-11-06 12:31:23,401 [htrace.SpanReceiverBuilder] ERROR: SpanReceiverBuilder cannot find SpanReceiver class org.apache.accumulo.tracer.ZooTraceClient: disabling span receiver.
2017-11-06 12:31:23,401 [trace.DistributedTrace] WARN : Failed to load SpanReceiver org.apache.accumulo.tracer.ZooTraceClient
2017-11-06 12:31:23,989 [shell.Shell] ERROR: org.apache.accumulo.core.client.AccumuloException: File C00001tr.rf does not exist in import dir
2017-11-06 12:31:25,051 [conf.ConfigSanityCheck] WARN : Use of instance.dfs.uri and instance.dfs.dir are deprecated. Consider using instance.volumes instead.
2017-11-06 12:31:25,774 [htrace.SpanReceiverBuilder] ERROR: SpanReceiverBuilder cannot find SpanReceiver class org.apache.accumulo.tracer.ZooTraceClient: disabling span receiver.
2017-11-06 12:31:25,774 [trace.DistributedTrace] WARN : Fa

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/accumulo-1.8.1/lib/slf4j-log4j12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/accumulo-1.8.1/lib/slf4j-log4j12.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


# Setup Datastores

In [ ]:
%%bash
# clear out potential old runs
geowave config rmstore kmeans_gpx
geowave config rmstore germany_gpx_accumulo

# configure geowave connection params for name stores "germany_gpx_accumulo" and "kmeans_hbase"
geowave config addstore germany_gpx_accumulo --gwNamespace geowave.germany_gpx -t accumulo --zookeeper $HOSTNAME:2181 --instance accumulo --user root --password secret
geowave config addstore kmeans_gpx --gwNamespace geowave.kmeans -t hbase --zookeeper $HOSTNAME:2181

# Run KMeans

In [ ]:
%%bash

geowave remote clear kmeans_gpx

In [ ]:
#grab classes from jvm
hbase_options_class = sc._jvm.mil.nga.giat.geowave.datastore.hbase.operations.config.HBaseRequiredOptions
accumulo_options_class = sc._jvm.mil.nga.giat.geowave.datastore.accumulo.operations.config.AccumuloRequiredOptions
kmeans_runner_class = sc._jvm.mil.nga.giat.geowave.analytic.javaspark.kmeans.KMeansRunner
query_options_class = sc._jvm.mil.nga.giat.geowave.core.store.query.QueryOptions
geowave_rdd_class = sc._jvm.mil.nga.giat.geowave.analytic.javaspark.GeoWaveRDD
sf_df_class = sc._jvm.mil.nga.giat.geowave.analytic.javaspark.sparksql.SimpleFeatureDataFrame
byte_array_class = sc._jvm.mil.nga.giat.geowave.core.index.ByteArrayId

In [ ]:
#setup input datastore
input_store = accumulo_options_class()
input_store.setInstance('accumulo')
input_store.setUser('root')
input_store.setPassword('secret')
input_store.setZookeeper(os.environ['HOSTNAME'] + ':2181')
input_store.setGeowaveNamespace('geowave.germany_gpx')

#Setup output datastore
output_store = hbase_options_class()
output_store.setZookeeper(os.environ['HOSTNAME'] + ':2181')
output_store.setGeowaveNamespace('geowave.kmeans')

#Create a instance of the runner
kmeans_runner = kmeans_runner_class()

input_store_plugin = input_store.createPluginOptions()
output_store_plugin = output_store.createPluginOptions()

In [ ]:
#set the appropriate properties
#We want it to execute using the existing JavaSparkContext wrapped by python.
kmeans_runner.setJavaSparkContext(sc._jsc)

kmeans_runner.setAdapterId('gpxpoint')
kmeans_runner.setNumClusters(8)
kmeans_runner.setInputDataStore(input_store_plugin)
kmeans_runner.setOutputDataStore(output_store_plugin)
kmeans_runner.setCqlFilter("BBOX(geometry,  13.3, 52.45, 13.5, 52.5)")
kmeans_runner.setCentroidTypeName('mycentroids')
kmeans_runner.setHullTypeName('myhulls')
kmeans_runner.setGenerateHulls(True)
kmeans_runner.setComputeHullData(True)
#execute the kmeans runner
kmeans_runner.run()

# Load Centroids into DataFrame and display

In [ ]:
# Create the dataframe and get a rdd for the output of kmeans
sf_df = sf_df_class(spark._jsparkSession)
adapter_id = byte_array_class('mycentroids')

queryOptions = None
adapterIt = output_store_plugin.createAdapterStore().getAdapters()
adapterForQuery = None
while (adapterIt.hasNext()):
    adapter = adapterIt.next()
    if (adapter.getAdapterId().equals(adapter_id)):
        adapterForQuery = adapter
        queryOptions = query_options_class(adapterForQuery)
        break

output_rdd = geowave_rdd_class.rddForSimpleFeatures(sc._jsc.sc(), output_store_plugin, None, queryOptions)

sf_df.init(output_store_plugin, adapter_id)

df = sf_df.getDataFrame(output_rdd)
# Convert Java DataFrame to Python DataFrame
import pyspark.mllib.common as convert
py_df = convert._java2py(sc, df)

py_df.createOrReplaceTempView('mycentroids')

df = sql_context.sql("select * from mycentroids")

display(df)

# Parse DataFrame data into lat/lon columns and display centroids on map
Using pixiedust's built in map visualization we can display data on a map assuming it has the following properties.
- Keys: put your latitude and longitude fields here. They must be floating values. These fields must be named latitude, lat or y and longitude, lon or x.
- Values: the field you want to use to thematically color the map. Only one field can be used.

Also you will need a access token from whichever map renderer you choose to use with pixiedust (mapbox, google).
Follow the instructions in the token help on how to create and use the access token.

In [ ]:
# Convert the string point information into lat long columns and create a new dataframe for those.
import pyspark
def parseRow(row):
    lat_start = row.geom.rfind(' ') + 1
    lat_end = row.geom.rfind(')')
    lat = row.geom[lat_start:lat_end]
    lon_start = row.geom.find('(') + 1
    lon_end = row.geom.rfind(' ', lon_start)
    lon = row.geom[lon_start:lon_end]
    return pyspark.sql.Row(lat=float(lat), lon=float(lon), ClusterIndex=row.ClusterIndex)
    
row_rdd = df.rdd
new_rdd = row_rdd.map(lambda row: parseRow(row))
new_df =new_rdd.toDF() 
display(new_df)

# Export KMeans Hulls to DataFrame
If you have some more complex data to visualize pixiedust may not be the best option.

The Kmeans hull generation outputs polygons that would be difficult for pixiedust to display without
creating a special plugin. 

Instead, we can use another map renderer to visualize our data. For the Kmeans hulls we will use ipyleaflet to visualize the data. We will start by grabbing the results for the hull generation and putting them into a DataFrame

In [ ]:
# Create the dataframe and get a rdd for the output of kmeans
sf_df_hulls = sf_df_class(spark._jsparkSession)
adapter_id = byte_array_class('myhulls')

queryOptions = None
adapterIt = output_store_plugin.createAdapterStore().getAdapters()
adapterForQuery = None
while (adapterIt.hasNext()):
    adapter = adapterIt.next()
    if (adapter.getAdapterId().equals(adapter_id)):
        adapterForQuery = adapter
        queryOptions = query_options_class(adapterForQuery)
        break

output_rdd_hulls = geowave_rdd_class.rddForSimpleFeatures(sc._jsc.sc(), output_store_plugin, None, queryOptions)

sf_df_hulls.init(output_store_plugin, adapter_id)

df_hulls = sf_df_hulls.getDataFrame(output_rdd_hulls)
# Convert Java DataFrame to Python DataFrame
import pyspark.mllib.common as convert
py_df_hulls = convert._java2py(sc, df_hulls)

py_df_hulls.createOrReplaceTempView('myhulls')

df_hulls = sql_context.sql("select * from myhulls order by Density")

display(df_hulls)

# Convert Kmeans hull results to geojson
ipyleaflet provides an easy way to visualize leaflet maps in jupyter notebooks.

Our hull data contains wkt geometry strings that we will use with a small python library to convert the geometry to GeoJson. Once our data is converted to a proper GeoJson feature collection we can use ipyleaflet to easily load and display that data on a map.

For more information on the GeoJson format visit: http://geojson.org/

In [ ]:
from geomet import wkt
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

# Collecting the results will give a array of Rows.
hulls_results = df_hulls.collect()
hulls_geojson = {
    "type": "FeatureCollection",
    "features": []
}
for hull in hulls_results:
    hull = hull.asDict(True)
    output_geojson = {
        "type": "Feature",
        "geometry": {},
        "properties": {}
    }
    # Convert geometry to geojson with geomet
    geom = wkt.loads(hull["geom"])
    output_geojson["geometry"] = geom
    for propKey in hull:
        if propKey != "geom":
            output_geojson["properties"][propKey] = hull[propKey]
    hulls_geojson["features"].append(output_geojson)
print("Count: {0} Features".format(len(hulls_geojson["features"])))

In [ ]:
center = [52.54, 13.49]
zoom = 10

In [ ]:
m = Map(center=center, zoom=zoom)
g = GeoJSON(data=hulls_geojson)
m.add_layer(g)
m